In [1]:
import dash
from dash import html, dcc, Output, Input, State
import dash_leaflet as dl
import plotly.express as px
import pandas as pd

# Datos dummy
data_dummy = pd.DataFrame({
    'Ciudad': ['CDMX', 'CDMX', 'CDMX', 'CDMX', 'CDMX'],
    'Colonia': ['Coyoacán', 'Coyoacán', 'Condesa', 'Polanco', 'Centro'],
    'Nombre': ['Studio A', 'Studio B', 'Studio C', 'Studio D', 'Studio E'],
    'Tipo de Estudio': ['Yoga', 'Crossfit', 'Pilates', 'Gimnasio', 'Danza'],
    'Fit Pass Pro': ['Sí', 'No', 'Sí', 'No', 'Sí'],
    'Latitud': [19.4326, 19.4312, 19.4298, 19.4284, 19.4270],
    'Longitud': [-99.1332, -99.1318, -99.1304, -99.1290, -99.1276],
    'Cantidad': [10, 5, 8, 12, 7] 
})

# Gráfico de barras
fig = px.bar(data_dummy, x='Tipo de Estudio', y='Cantidad', color='Fit Pass Pro', title='Número de Estudios por Tipo')

# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Layout de la aplicación
app.layout = html.Div([
    html.H1("Dashboard de Fitpass", style={'color': '#007bff'}),
    dcc.Dropdown(id='dropdown-ciudad', options=[{'label': c, 'value': c} for c in data_dummy['Ciudad'].unique()], value='CDMX'),
    dcc.Dropdown(id='dropdown-colonia', value='Coyoacán'),
    html.Button('Buscar', id='btn-buscar', style={'background-color': '#28a745', 'color': 'white'}),
    html.Div(id='lista-estudios', style={'padding': '20px'}),
    dl.Map(center=[19.4326, -99.1332], zoom=12, id='map', style={'width': '1000px', 'height': '500px'}),
    dcc.Graph(figure=fig)
])

# Callback para actualizar la lista de estudios y el mapa
@app.callback(
    [Output('lista-estudios', 'children'), Output('map', 'children')],
    [Input('btn-buscar', 'n_clicks')],
    [State('dropdown-ciudad', 'value'), State('dropdown-colonia', 'value')]
)
def actualizar_lista_y_mapa(n_clicks, ciudad, colonia):
    if n_clicks is None:
        return [html.P('Seleccione una ciudad y colonia para buscar.'), [dl.TileLayer()]]

    # Aquí debemos agregar la lógica para actualizar la lista y el mapa
    # ...

# Corre app de Dash
if __name__ == '__main__':
    app.run_server(debug=True)



ImportError: cannot import name 'get_current_traceback' from 'werkzeug.debug.tbtools' (/Users/ravj/opt/anaconda3/envs/fitpass-env/lib/python3.9/site-packages/werkzeug/debug/tbtools.py)